<a href="https://colab.research.google.com/github/ascle/colab_tcc2/blob/dev/6_melhorando_modelo_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importando bibliotecas

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

### Outras configurações<hr style='border: 2px solid red;'>

In [2]:
pd.set_option('display.max_columns', None)
SEED = 7
np.random.seed(SEED)

### Conectando no ambiente que será utilizado

In [3]:
# Importando do Drive
from google.colab import drive
from os import chdir

drive.mount('/content/drive', force_remount=True)
chdir('/content/drive/MyDrive/UFS/tcc-2-ascle-ufs/bases/linkage_marco/')

Mounted at /content/drive


### Leitura dos dados <hr style='border: 2px solid red;'>

In [4]:
data_raw = pd.read_csv('dn_etl.csv', sep=';', low_memory=False)
data_raw.sort_index(axis=1, inplace=True)

In [5]:
data_raw.shape

(192484, 24)

In [6]:
data_raw.head()

,ANO,APGAR1,APGAR5,CODANOMAL_QTD,CONSPRENAT,ESCMAEETL_VALOR,ESTCIVMAE_COMPANHEIRO,GRAVIDEZ_VALOR,LOCNASC_DOMICILIO,LOCNASC_ESTAB_SAUDE,LOCNASC_HOSPITAL,LOCNASC_OUTROS,OBT_NEONATAL,PARTO_CESAREO,PARTO_VAGINALL,PESO,QTDFILMORT,QTDPARTCES,SEMAGESTAC,SEXO_FEM,SEXO_MASC,TPAPRESENT_CEFALICO,TPAPRESENT_PELVICO,TPAPRESENT_TRANSVERSA
0,2015,2.0,4.0,0.0,5.0,1,0,1,0,0,1,0,0,1,0,820.0,1.0,NaN,26.0,0,1,1,0,0
1,2015,9.0,10.0,0.0,8.0,4,1,1,0,0,1,0,0,1,0,4200.0,1.0,NaN,39.0,0,1,1,0,0
2,2015,7.0,8.0,0.0,10.0,11,1,1,0,0,1,0,0,0,1,3700.0,0.0,0.0,40.0,1,0,1,0,0
3,2015,7.0,8.0,0.0,7.0,4,0,1,0,0,1,0,0,0,1,3550.0,1.0,0.0,37.0,0,1,1,0,0
4,2015,9.0,10.0,0.0,3.0,11,0,1,0,0,1,0,0,1,0,3800.0,0.0,NaN,39.0,1,0,1,0,0


## Dados X e y<hr style='border: 2px solid red;'>

In [7]:
X_raw= data_raw.drop(columns=['QTDPARTCES', 'ANO', 'OBT_NEONATAL'])
y_raw= data_raw["OBT_NEONATAL"]

## Dados desbalanceados<hr style='border: 2px solid red;'>

In [8]:
from imblearn.over_sampling import SMOTE

In [9]:
smt = SMOTE(random_state=SEED)
X_bal, y_bal = smt.fit_resample(X_raw, y_raw)

##Dados de treino e de teste<hr style='border: 2px solid red;'>

In [10]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X_bal, y_bal, test_size =0.3,
                                                        random_state=SEED,
                                                        stratify=y_bal)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(X_treino), len(X_teste)))

Treinaremos com 267512 elementos e testaremos com 114648 elementos


## Random Forest

In [11]:
modelo_rfc = RandomForestClassifier(n_estimators = 100)
modelo_rfc.fit(X_treino, y_treino)
predito_rfc = modelo_rfc.predict(X_teste)

In [12]:
print("A acurácia do modelo Random Forest foi %.2f%%" % (metrics.accuracy_score(y_teste, predito_rfc) * 100))
print("A Precisão do modelo Random Forest foi %.2f%%" % (metrics.precision_score(y_teste, predito_rfc) * 100))
print("A Recall do modelo Random Forest foi %.2f%%" % (metrics.recall_score(y_teste, predito_rfc) * 100))
print("A F1 do modelo Random Forest foi %.2f%%" % (metrics.f1_score(y_teste, predito_rfc) * 100))

A acurácia do modelo Random Forest foi 99.74%
A Precisão do modelo Random Forest foi 99.82%
A Recall do modelo Random Forest foi 99.67%
A F1 do modelo Random Forest foi 99.74%


### K-Best <hr style='border: 2px solid red;'>

In [13]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [27]:
df_kbest = pd.DataFrame(columns=['K', 'Acurácia', 'Precisão', 'Recall', 'F1'])

In [31]:
count = 1
for count in range(1, X_raw.shape[1]):
  selecionar_kmelhores = SelectKBest(chi2, k=count)
  selecionar_kmelhores.fit(X_treino, y_treino)

  X_treino_kbest = selecionar_kmelhores.transform(X_treino)
  X_teste_kbest = selecionar_kmelhores.transform(X_teste)

  modelo_kb = RandomForestClassifier(n_estimators = 100)
  modelo_kb.fit(X_treino_kbest, y_treino)
  predito_kb = modelo_kb.predict(X_teste_kbest)

  metricas = [count,
              (metrics.accuracy_score(y_teste, predito_kb) * 100),
              (metrics.precision_score(y_teste, predito_kb) * 100),
              (metrics.recall_score(y_teste, predito_kb) * 100),
              (metrics.f1_score(y_teste, predito_kb) * 100)]

  df_kbest.loc[count] = metricas


In [32]:
df_kbest

,K,Acurácia,Precisão,Recall,F1
1,1.0,95.878690,98.171994,93.498360,95.778197
2,2.0,99.255111,99.515976,98.991696,99.253144
3,3.0,99.497593,99.636135,99.358035,99.496891
4,4.0,99.645000,99.701357,99.588305,99.644799
5,5.0,99.690357,99.732868,99.647617,99.690224
6,6.0,99.717396,99.760799,99.673784,99.717273
7,7.0,99.720013,99.759074,99.680762,99.719903
8,8.0,99.726118,99.760841,99.691229,99.726023
9,9.0,99.721757,99.760820,99.682506,99.721648
10,10.0,99.712162,99.752086,99.672040,99.712047


### RFE <hr style='border: 2px solid red;'>

In [18]:
from sklearn.feature_selection import RFE

In [19]:
selecionador_rfe = RFE(estimator = classificador, n_features_to_select = 5, step = 1)

NameError: ignored

In [ ]:
selecionador_rfe.fit(treino_x, treino_y)
treino_rfe = selecionador_rfe.transform(treino_x)
teste_rfe = selecionador_rfe.transform(teste_x)
classificador.fit(treino_rfe, treino_y)

In [ ]:
matriz_confusao = confusion_matrix(teste_y, classificador.predict(teste_rfe))
sns.heatmap(matriz_confusao, annot = True, fmt = "d").set(xlabel = "Predição", ylabel = "Real")
print("Resultado da classificação %.2f%%" % (classificador.score(teste_rfe, teste_y)* 100))

### Cross Validation <hr style='border: 2px solid red;'>

In [ ]:
from sklearn.feature_selection import RFECV

In [ ]:
selecionador_rfecv = RFECV(estimator = classificador, cv = 5, scoring = "accuracy", step = 1)
selecionador_rfecv.fit(treino_x, treino_y)
treino_rfecv = selecionador_rfecv.transform(treino_x)
teste_rfecv = selecionador_rfecv.transform(teste_x)
classificador.fit(treino_rfecv, treino_y)

In [ ]:
matriz_confusao = confusion_matrix(teste_y, classificador.predict(teste_rfecv))
sns.heatmap(matriz_confusao, annot = True, fmt = "d").set(xlabel = "Predição", ylabel = "Real")

print("Resultado da classificação %.2f%%" % (classificador.score(teste_rfecv, teste_y)* 100))

In [ ]:
plt.xlabel("Número de exames")
plt.ylabel("Acurácia")

plt.plot(range(1, len(selecionador_rfecv.cv_results_['mean_test_score'] ) + 1), selecionador_rfecv.cv_results_['mean_test_score'])
plt.show